# 1. 데이터 수집하기
---
모델에 학습을 시키기 위해서는 마스크를 쓴 사진과 쓰지 않은 사진 그리고 착용 유무에 따른 레이블 데이터 셋트가 필요하다. 아직은 이와 관련된 데이터 셋트가 공개되지 않은 것으로 판단되어서 직접 데이터 셋트를 만들기로 하였다.

## 1.1 얼굴 데이터 셋트 만들기 

직접 얼굴 데이터 셋트를 만들기 위해서 cvlib를 사용했다. cvlib는 파이썬에서 얼굴 인식을 사용하기 위해 사용되는 라이브러리이다. cvlib 라이브러리를 사용하기 위해서 `opencv-python`과 `tensorflow`, `cvlib` 라이브러리가 설치되어야 한다. `pip install opencv-ptyon tensorflow cvlib` 명령으로 이 라이브러리들을 한꺼번에 설치할 수 있다. 

### 1.1.1 라이브러리 임포트 하기
우선 이 프로젝트에서 사용할 라이브러리를 모두 임포트한다. 

In [ ]:
pip install opencv-python tensorflow

In [ ]:
pip install cvlib

In [25]:
import cv2

cap = cv2.VideoCapture(0)               # 0번 카메라 장치 연결 ---①
if cap.isOpened():                      # 캡쳐 객체 연결 확인
    while True:
        ret, img = cap.read()           # 다음 프레임 읽기
        if ret:
            cv2.imshow('camera', img)   # 다음 프레임 이미지 표시
            if cv2.waitKey(1) != -1:    # 1ms 동안 키 입력 대기 ---②
                break                   # 아무 키라도 입력이 있으면 중지
        else:
            print('no frame')
            break
else:
    print("can't open camera.")
cap.release()                           # 자원 반납
cv2.destroyAllWindows()

can't open camera.


In [26]:
import cv2
import cvlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from IPython.display import Image
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import array_to_img, load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

### 1.1.2 웹캠 연결하기
얼굴 데이터를 캡쳐하기 위해서 웹캠을 사용한다. cv2.VideoCatpure(0)를 사용해서 컴퓨터에 연결된 웹캠을 연결한다. 그리고 웹캠이 제대로 연결되었으면 isOpened()는 True가 리턴된다. 

In [27]:
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    raise Exception("웹캡이 연결되지 않았습니다.")

Exception: ignored

### 1.1.2 웹캠으로 사진 캡쳐하기
webcam.read()를 사용하여 웹캠으로 사진을 캡쳐할 수 있다. read() 함수는 두개의 값을 튜플로 리턴한다. 첫 번째 값은 캡쳐가 되었으면 True, 안되었으면 False가 되고, 두 번째 값은 캡쳐된 이미지를 리턴한다. 

In [ ]:
is_readed, frame = webcam.read()
if not is_readed:
    raise Exception("캡쳐가 제대로 되지 않았습니다.")

Exception: ignored

### 1.1.3 얼굴 탐지하기
cvlib.detect_face() 함수를 사용하면 얼굴을 탐지할 수 있다. detect_face() 함수에 이미지를 전달하면 두개의 값을 튜플로 리턴된다. 첫 번째 값은 얼굴의 위치를 나타내고, 두 번째 값은 얼굴일 확률을 나타낸다. 

In [ ]:
faces, confidences = cvlib.detect_face(frame)
print(faces)
print(confidences)

TypeError: ignored

### 1.1.4 이미지 저장하기
cv2.imwrite() 함수를 사용하면 이미지를 저장할 수 있다. 첫 번째 인자는 저장할 파일의 위치와 파일이름을 전달하고, 두 번째 인자는 저장할 이미지를 전달한다. cvlib.detect_face() 함수로 찾은 얼굴 위치를 사용해서 얼굴만 추출하여 저장한다. 

In [ ]:
start_x, start_y, end_x, end_y = faces[0]
cv2.imwrite('1.jpg', frame[start_y:end_y, start_x:end_x, :])

### 1.1.5 웹캠으로 촬영 후 얼굴만 추출하여 저장하는 함수 만들기
1.1.1 ~ 1.1.4의 코드를 이용해서 웹캠으로 촬영 후 얼굴만 추출하여 저장하는 함수로 만들었다. 이 함수를 이용해서 마스크를 쓴 얼굴 사진 300장과 마스크를 쓰지 않은 얼굴 사진 300장을 만들었다. 

In [ ]:
import time
import cv2
import cvlib


def capture_n_save_face(path, max=1):
    count = 0
    
    # 웹캠 연결하기
    webcam = cv2.VideoCapture(0)
    if not webcam.isOpened():
        raise Exception("웹캡이 연결되지 않았습니다.")
        
    while count < max:
        time.sleep(0.3)   # 다양한 얼굴이 저장될 수 있도록 0.3초 지연을 준다. 
        is_readed, frame = webcam.read() # 웹캠으로 촬영하기 

        if not is_readed:
            raise Exception("캡쳐가 제대로 되지 않았습니다.")

        faces, confidences = cvlib.detect_face(frame)   # 얼굴 탐지하기

        for face, confidence in zip(faces, confidences):
            if confidence < 0.8:   # 얼굴일 확률이 80% 이하면 패스한다.
                continue
            start_x, start_y, end_x, end_y = face
            try:
                cv2.imwrite(path+str(count)+'.jpg', frame[start_y:end_y, start_x:end_x, :])   # 얼굴만 저장하기 
                count += 1
            except:
                break
        
        print(count, end=' ')

    webcam.release()


# capture_n_save_face('c:\\non_mask\\', 300) # 마스크를 쓰지 않은 사진 300만 만들기
# capture_n_save_face('c:\\mask\\', 300) # 마스크를 쓴 사진 300장 만들기


### 1.1.6 만들어진 사진 확인하기
captur_n_save_face() 함수를 사용하여 다양한 각도로 사진이 캡쳐될 수 있도록 촬영하였다.

In [ ]:
Image("non_mask_images.png")

In [ ]:
Image("mask_images.png")

# 2. 이미지 크기 파악하기
---
딥러닝 모델에 데이터를 학습하기 위해서 통일된 데이터의 크기로 전처리를 해야 한다. 그래서 전처리 전에 모든 이미지 데이터의 크기를 그래프를 사용하여 확인한 후 적절한 크기를 찾아보겠다. 

## 2.1 이미지 목록 불러오기

이미지를 불러 오기 전에 우선 이미지의 이름을 알아야 한다. os.listdir() 함수를 사용하면 디렉터리 안에 있는 파일 리스트를 얻을 수 있다. 마스크를 쓰지 않는 이미지는 c:\non_mask\에 마스크를 쓴 이미지는 c:\mask\에 모아두었다.

In [ ]:
non_mask_list = os.listdir('c:\\non_mask\\')
mask_list = os.listdir('c:\\mask\\')

## 2.2 이미지 목록을 이용하여 이미지 크기 얻기

cv2.imread()로 이미지를 불러온 후 shape 속성에 접근하면 (세로 * 가로 * 채널) 정보를 튜플로 얻을 수 있다. 

In [ ]:
image = cv2.imread("c:\\non_mask\\" + non_mask_list[0])
image.shape

이제 모든 이미지의 shape 속성을 이용해서 가로 정보를 가지는 widths 리스트와 세로 정보를 가지는 heights 리스트를 만들어 보자.

In [ ]:
widths = []
heights = []

for non_mask in non_mask_list:
    image = cv2.imread("c:\\non_mask\\" + non_mask)
    widths.append(image.shape[1])
    heights.append(image.shape[0])
    
for mask in mask_list:
    image = cv2.imread("c:\\mask\\" + mask)
    widths.append(image.shape[1])
    heights.append(image.shape[0])

## 2.3 이미지 크기를 그래프로 확인하기

이미지 크기 분포를 보기 좋게 하기 위해서 그래프를 이용한다. 

### 2.3.1 삼전도로 출력하기

matploglib 라이브러리를 사용하여 가로, 세로로 삼전도를 출력해보니 대부분 일정 비율로 이미지 크기가 이루어지는 것을 확인 할 수 있다.

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(widths, heights, alpha=0.3)

### 2.3.2 히스토그램으로 출력하기

#### 가로 크기 히스토그램

가로 크기를 히스토그램으로 나타내어 보자.

In [ ]:
cut_widths = pd.cut(widths, np.arange(50, 260, 10)).value_counts()

fig, ax = plt.subplots(figsize=(26, 7))
ax.bar(range(len(cut_widths)),cut_widths.values)
ax.set_xticks(range(len(cut_widths)))
ax.set_xticklabels(cut_widths.index)
ax

#### 세로 크기 히스토그램

동일한 방법으로 세로 크기를 히스토그램을 나타내어 보자.

In [ ]:
cut_heights = pd.cut(heights, np.arange(50, 260, 10)).value_counts()

fig, ax = plt.subplots(figsize=(25,7))
ax.bar(range(len(cut_heights)),cut_heights.values)
ax.set_xticks(range(len(cut_heights)))
ax.set_xticklabels(cut_heights.index)
ax

### 2.3.3 모델에 사용할 이미지 크기 결정하기 
모델에 학습을 시키기 위해서는 같은 크기로 만들어야 한다. 너무 작은 이미지를 크게 만들게 되면 정보 손실이 발생하고 너무 큰 이미지를 작게 만들면 정보량이 부족해 진다. 그래서 위의 그래프를 참고해서 적절한 크기를 찾아야 한다.

가로는 90\~190, 세로는 90\~240사이에 데이터가 많이 모여 있는 것을 볼 수 있다. 그래서 중간값이라고 생각되는 크기를 사용해서 가로는 140, 세로는 180을 사용하겠다. 

# 3. 데이터 전 처리
---


## 3.1 통일된 이미지 크기로 이미지 가져오기

이제 통일된 이미지 크기로 이미지를 가져오겠다. 이미지는 위에 결정한대로 가로 140, 세로 180으로 크기를 통일시킨다. 그리고 이미지를 가져올 때 마스크를 쓴 사진의 라벨은 1로 하고, 마스크를 쓰지 않은 사진의 라벨은 0으로 구분한다. 

In [ ]:
image_height = 180
image_width = 140

images = []
labels = []

for non_mask in non_mask_list:
    image = load_img('c:\\non_mask\\'+non_mask, target_size=(image_height, image_width))
    image = img_to_array(image)
    images.append(image)
    labels.append(0)
    
for mask in mask_list:
    image = load_img('c:\\mask\\'+mask, target_size=(image_height, image_width))
    image = img_to_array(image)
    images.append(image)
    labels.append(1)

images[0]의 shape를 출력하니 세로 180, 가로 140, 채널 3인 것을 확인 할 수 있다. 

In [ ]:
images[0].shape

## 3.2 데이터 분리하기

가져온 이미지를 sklearn.model_selection에서 train_test_split()를 사용하여 훈련용 데이터와 검증용 데이터, 테스트용 데이터를 8:1:1 비율로 나눈다. 그리고 분리시 마스크를 쓴 사진과 마스크를 쓰지 않은 사진이 섞이도록 한다. 

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(np.array(images), np.array(labels), test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)

y_train을 출력해보니 사진이 잘 섞인 것을 확인 할 수 있다. 

In [ ]:
y_train

# 4. 딥러닝 모델 만들고 학습하기

---

## 4.1 CNN 모델 설정

이제 딥러닝에 사용할 CNN 모델을 구현한다. 모델의 층 구성은 AI 온라인 교육 내용을 참고하여 만들었다. 

In [ ]:
model = Sequential([    
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(image_height, image_width, image_channel)),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(rate=0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.25),
    Dense(2, activation='softmax')
])

model.summary()

## 4.2 모델 컴파일하기

모델의 출력은 마스크를 썼는지 안 썼는지를 나타내야 하기 때문에 loss 함수는 sparse_categorical_crossentropy로 설정하고 optimizer는 adam을 사용하게 하겠다. 

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

## 4.3 모델 학습시키기

이렇게 만들진 모델에 학습 데이터와 검증 데이터를 사용하여 학습시킨다. 

In [ ]:
EPOCHS = 30

history = model.fit(x_train, 
                    y_train,
                    validation_data = (x_val, y_val),
                    epochs=EPOCHS, 
                   )

모델을 학습하면서 출력되는 손실값과 정확도를 확인할 수 있다. 15에포크부터는 검증 데이터의 정확도가 100%인 것을 확인 할 수 있다. 

## 4.4 학습을 수행시 Accuracy와 Loss 변화 그래프로 확인하기



In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 4.5 모델 성능 평가 및 예측

학습이 잘 되었는지 평가용 데이터로 평가를 해보자. 

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)

print('test set accuracy: ', test_accuracy)

정확도가 100%로 나옵니다. 
실제 이미지를 출력해서 실제 라벨과 예측값을 출력해보자. 

In [ ]:
test_prediction = np.argmax(model.predict(x_test), axis=-1)

In [ ]:
plt.figure(figsize = (13, 13))

start_index = 0
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    prediction = test_prediction[start_index + i]
    actual = y_test[start_index + i]
    col = 'g'
    if prediction != actual:
        col = 'r'
    plt.xlabel('Actual={} || Pred={}'.format(actual, prediction), color = col)
    plt.imshow(array_to_img(x_test[start_index + i]))
plt.show()

마지막으로 confusion matrix를 시각화하여 분류 학습 결과를 확인해 보자. 29개의 마스크를 쓰지 않은 이미지와 31개의 마스크를 쓴 이미지를 정확하게 예측한 것을 확인할 수 있다. 

In [ ]:
cm = confusion_matrix(y_test, test_prediction)
sns.heatmap(cm, annot = True)

# 5. 웹캠을 사용하여 실시간으로 마스크 착용 유무 확인하기

---

이제 웹캠을 사용하여 실시간으로 마스크 착용 유무를 확인하려고 한다. 실시간으로 확인을 하기 위해서 opencv를 사용할 것이다.


## 5.1 모델 저장하기 

지금까지 학습한 모델을 저장할 수 있다. model에 save() 메서드를 사용하면 모델을 저장할 수 있다. 

In [ ]:
model.save('detect_face_model.h5')

## 5.2 모델 불러오기

저장한 모델은 load_model() 함수를 사용하여 불러올 수 있다. 이렇게 불러온 모델은 이미 학습이 되어 있는 상태이므로 바로 입력 데이터를 사용해서 예측이 가능하다. 그리고 불러온 모델은 summary()를 통해서 모델이 어떻게 구성되어 있는지 확인할 수 있다. 

In [ ]:
model = load_model('detect_face_model.h5')
model.summary()

## 5.3. 웹캠으로 캡쳐한 얼굴 예측해보기

웹캠으로 캡쳐한 얼굴을 예측하기 위해서는 학습을 하기 위해서 사용했던 데이터와 동일한 방법으로 전처리를 해야 한다. 대부분의 코드는 데이터 수집에서 사용한 코드와 동일하다. 하지만 학습에 사용한 데이터들을 여러개의 이미지를 입력하였지만, 실시간으로 캡쳐한 데이터는 1개라는 점이 다르다. 그래서 캡쳐한 데이터는 1개의 데이터를 가지는 배열로 변환시켜야 한다. 

배열로 변화시키기 위해서 np.expand_dims()를 사용한다. 

In [ ]:
faces, confidences = cvlib.detect_face(frame)   # 얼굴 탐지하기
start_x, start_y, end_x, end_y = face[0]

face_frame = frame[start_y:end_y, start_x:end_x, :]   # 얼굴 영역

cv2.imwrite('temp.jpg', face_frame)
x = load_img('temp.jpg', target_size=(image_height, image_width))
print("origin_x.shape: ", x.shape)

x = img_to_array(x)
x = np.expand_dims(x, axis=0)
print("expand_dims_x.shape: ", x.shape)

pre = model.predict(x)
is_wear_mask = np.argmax(pre)
print("model.predict(): ", pre)
print("is_wear_mask: ", is_wear_mask")

## 5.4 최종 프로그램 작성

위의 내용을 사용해서 웹캠을 사용하여 실시간으로 마스크 착용 유무를 확인하는 프로그램을 만들어 보자. 이 코드는 예측한 결과를 이미지에 얼굴 영역을 네모 박스로 그리고 마스크 착용 여부를 출력하기 위해서 cv2.imshow()를 사용한다. cv2.imshow()는 주피터 노트북에서 제대로 작동하지 않기 때문에 파이썬 코드로 따로 실행을 시켜야 한다. 이 프로그램은 q 키보드를 누르기 전까지 계속 실행되게 만들었다. 

In [ ]:
Image("mask_detect_program.png")

In [ ]:
import cvlib
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import ImageFont, ImageDraw, Image

image_height = 180
image_width = 140

model = load_model('detect_face_model.h5')  # 학습한 모델 불러오기

# 웹캠 연결하기
webcam = cv2.VideoCapture(0)
if not webcam.isOpened():
    raise Exception("웹캡이 연결되지 않았습니다.")

while webcam.isOpened():
    is_readed, frame = webcam.read()  # 웹캠으로 촬영하기

    if not is_readed:
        raise Exception("캡쳐가 제대로 되지 않았습니다.")

    faces, confidences = cvlib.detect_face(frame)   # 얼굴 탐지하기

    for face, confidence in zip(faces, confidences):
        if confidence < 0.8:   # 얼굴일 확률이 80% 이하면 패스한다.
            continue
        start_x, start_y, end_x, end_y = face

        try:
            face_frame = frame[start_y:end_y, start_x:end_x, :]   # 얼굴 영역 추출하기

            cv2.imwrite('temp.jpg', face_frame)
            x = load_img('temp.jpg', target_size=(image_height, image_width))

            x = img_to_array(x) 
            x = np.expand_dims(x, axis=0)   # 예측을 위해 배열로 만들기

            pre = model.predict(x)

            is_wear_mask = np.argmax(pre)   # 마스크 착용 여부 결정 - 1: 마스크 착용, 0: 마스크 미착용
            
            # 마스크 착용 미착용에 따라 색상 및 문구를 설정
            print_text = 'No mask'
            color = (0, 0, 255)
            if is_wear_mask:
                print_text = 'Mask'
                color = (0, 255, 0)
                
            # 캡처한 이미지에 얼굴 영역을 네모 박스와 마스크 착용 문구를 출력한다.
            cv2.rectangle(frame, (start_x, start_y),
                          (end_x, end_y), color, 2)
            y = start_y - 10 if start_y - 10 > 10 else start_y + 10
            text = print_text
            cv2.putText(frame, text, (start_x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        except:
            break
    
    cv2.imshow("Mask detect", frame)   # 마스크 착용 여부를 볼 수 있는 창을 연다. 

    if cv2.waitKey(1) & 0xFF == ord('q'):   # q를 입력시 프로그램이 종료된다. 
        break

cv2.destroyAllWindows()
webcam.release()